# Forensics - Create VHD Snapshot

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ Azure 4.0.0<br>
__OS Supported:__<br>
    -  Linux
    -  Windows
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to create Azure IaaS virtual machine snapshots for managed disks.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Prerequisite check

In [ ]:
# only run once, current version 0.1.2
!pip install --upgrade Sentinel-Utilities

In [ ]:
import SentinelUtils
# checking Python version
check = SentinelUtils.version_management.ModuleVersionCheck()
py_check = check.validate_python('3.6.0')
if py_check.requirement_met == False:
    print('Please select Python 3.6 or Python 3.6 - AzureML at the upper right corner')
else:
    print('Please continue')

In [ ]:
# checking required packages
mods_check = check.validate_installed_modules(['Azure>=4.0.0'])
for mod_info in mods_check:
    if mod_info.requirement_met == False:
        print('Please install {} {} at the following cell.'.format(mod_info.name, mod_info.required_version))

In [ ]:
# Please install required packages based on the check at last cell
#!pip install azure

## Table of Contents

1. Retrieve Azure Resource Information
2. Authentication to Azure Resource Management 
3. Create Snapshot and Generate SAS URL

## 1. Retrieve Azure Resource Information

In [ ]:
path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True

# information from config.json
if dsvm == False:
    # Run this if you are using Free Compute
    tenant_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[0]
    subscription_id = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[1]
    resource_group = SentinelUtils.config_reader.ConfigReader.read_config_values("config.json")[2]
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')

## 2. Authentication to Azure Resource Management

In [ ]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import DiskCreateOption

1. Select Authentication Methods<br>
If multi-factors authentication is enabled for your tenant, you must use Service Principal to authentication to Azure resource management.  Otherwise, you may choose either Service Principal or User ID/Password.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
auth_method = ['Service Principal', 'User ID/Password']
selected_auth = widgets.Dropdown(options=auth_method, value=auth_method[0],description='Auth Method:')
display(selected_auth)

2. Authentication

In [ ]:
if selected_auth.value == 'Service Principal':
    credentials = ServicePrincipalCredentials(
        client_id=input('client_id:'), 
        secret=input('secret:'), 
        tenant=tenant_id)
else:
    from azure.common.credentials import UserPassCredentials
    import getpass
    uid = input('User ID:')
    pwd = getpass.getpass()
    credentials = UserPassCredentials(uid, pwd)
    
client = ResourceManagementClient(credentials, subscription_id)

## 3. Create Snapshot and Generate SAS URL

1. Initialization

In [ ]:
resource_group_client = ResourceManagementClient(credentials, subscription_id)
compute_client = ComputeManagementClient(credentials, subscription_id)
network_client = NetworkManagementClient(credentials, subscription_id)

In [ ]:
# User input
vm_name = input('vm_name:')
vm_location = input('vm_location:')
snapshot_name = input('Name to use for the new snapshot:')

2. Get a Managed Disk

In [ ]:
def get_vm_disk_names(compute_client):
    vm = compute_client.virtual_machines.get(resource_group, vm_name, expand='instanceView')
    
    if vm is not None and vm.instance_view is not None:
        return [d.name for d in vm.instance_view.disks]
    else:
        return []

In [ ]:
import ipywidgets as widgets
from IPython.display import display
disks = get_vm_disk_names(compute_client)
#tables = list(dbSchema.keys())
selected_disk = widgets.Dropdown(options=disks, value=disks[0],description='Disk:')
display(selected_disk)

In [ ]:
managed_disk = compute_client.disks.get(resource_group, selected_disk)

3. Create Snapshot

In [ ]:
async_snapshot_creation = compute_client.snapshots.create_or_update(
        resource_group,
        snapshot_name,
        {
            'location': managed_disk.location,
            'creation_data': {
                'create_option': 'Copy',
                'source_uri': managed_disk.id
            }
        }
    )

In [ ]:
snapshot = async_snapshot_creation.result()

4. Generate SAS URL for the snapshot

In [ ]:
async_snapshot_export = compute_client.snapshots.grant_access(
    resource_group,
    snapshot_name,
    'read',
    360000)

In [ ]:
result = async_snapshot_export.result()

In [ ]:
# Snapshot SAS URL:
result.access_sas